In [1]:
# -*- coding: utf-8 -*-
%load_ext autoreload
%autoreload 2

import os, time
# select GPU for training
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# import tensorflow.experimental.numpy as np
import numpy as np
import tensorflow as tf
import random
import scipy.io as scio
import h5py 
import math

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

In [3]:
import sys
sys.path.append("../../")
import lib
from lib.proc import *
from lib.simu import *
from lib.commons import *
from lib.net import *

In [4]:
LENGTH = 130
CV_SP = 0
reg = 1e-2
prefix = "T1W_ToftsModelFilt_l2-RES01-joint-mLossDC-len%s-CV%s-RoQ-XAIFinput"%(str(LENGTH),str(CV_SP))
ISTRAIN = True
sample_pattern = 'RES01'
# parameters to set
DCE_DIM = 360
INIT_DIM = 4

OUT_DIM = LENGTH
IN_DIM = OUT_DIM
PK_DIM = 2

BATCH_SIZE = 1 # 
LR_RATE = 1e-3

BUFFER_SIZE = 8*BATCH_SIZE  # buffer size for shuffle
N_EPOCHS = 200
CKPT_PERIOD = 1

LAYER_NUM = 4
FEATURE_NUM = 64

hct=0.4

print(prefix)

T1W_ToftsModelFilt_l2-RES01-joint-mLossDC-len130-CV0-RoQ-XAIFinput


In [5]:
mainfolder = "/hdd1/chaowei/data/dce/pancreas_syn_purified_linCA_wt_bdTK_VARPRO/"
train_ind,test_ind = get_train_test_index(CV_SP,10)
print(train_ind,test_ind)


[1, 2, 3, 4, 5, 6, 7, 8, 9] 0


In [6]:
# load as array
X_train_ts = np.zeros([0,IN_DIM])
Y_train_ts = np.zeros([0,IN_DIM])
X_train_pl = np.zeros([0,IN_DIM])
Y_train_pl = np.zeros([0,IN_DIM])

for j in train_ind:
    foldername = os.path.join(mainfolder,str(j))
    case_id = os.listdir(foldername)
    for i in case_id:
        print('processing:',i) 
        h5file = h5py.File(os.path.join(foldername,i),'r')
        if np.array(h5file['c_ts']).ndim is 1:
            continue
        kv = np.array(h5file['kv'],dtype=np.float32).transpose(1,0)

        C_ts =  np.concatenate((np.array(h5file['c_ts'],dtype=np.float32).transpose(1,0),
                                np.array(h5file['c_bds'],dtype=np.float32).transpose(1,0)),axis=0)
        C_ts_lbl=C_ts
        
        C_bd =  np.mean(np.array(h5file['c_pl_ori'],dtype=np.float32).transpose(1,0),axis=0,keepdims=True)
        C_bd_lbl =  C_bd
        norm_constant = np.max(C_bd_lbl)*(1-hct)

        # generate downsampled R1
        C_ts_ds,C_ts,bundle = gen_ds_R1(C_ts,INIT_DIM,C_bd_lbl,OUT_DIM=OUT_DIM,sample_pattern=sample_pattern,average=False)
        # generate blood input 
        C_bd_ds,C_bd,bundle = gen_ds_R1(C_bd,INIT_DIM,C_bd_lbl,OUT_DIM=OUT_DIM,sample_pattern=sample_pattern,average=False)
        # clip label 
        _,C_ts_lbl,_ = gen_ds_R1(C_ts_lbl,INIT_DIM,C_bd_lbl,OUT_DIM=OUT_DIM,sample_pattern=sample_pattern,average=False)
        _,C_bd_lbl,_ = gen_ds_R1(C_bd_lbl,INIT_DIM,C_bd_lbl,OUT_DIM=OUT_DIM,sample_pattern=sample_pattern,average=False)
        # generate spike
        C_ts_itnpl = gen_inge_spike(C_ts_ds,bundle)
        C_bd_itnpl = gen_inge_spike(C_bd_ds,bundle)

        bd_temp = np.repeat(C_bd_itnpl.reshape([-1,OUT_DIM]),C_ts.shape[0],axis=0)
        bd_temp2 = np.repeat(C_bd.reshape([-1,OUT_DIM]),C_ts.shape[0],axis=0)
    
        X_train_ts = np.append(X_train_ts,C_ts_itnpl/norm_constant,axis=0)
        X_train_pl = np.append(X_train_pl,bd_temp/norm_constant,axis=0)
        Y_train_ts = np.append(Y_train_ts,C_ts/norm_constant,axis=0)
        Y_train_pl = np.append(Y_train_pl,bd_temp2/norm_constant,axis=0)
        h5file.close()
        
print('Train data loaded. X_train_ts Shape:',X_train_ts.shape,'X_train_pl Shape:',X_train_ts.shape)


processing: HP_0508_z16_pan.mat


<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-6-aa8a91984345>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if np.array(h5file['c_ts']).ndim is 1:


processing: PS_2216_z36_pan.mat
processing: HP_0522_z18_pan.mat
processing: PS_2216_z32_pan.mat
processing: PD_2084_z23_tum.mat
processing: PD_2084_z15_pan.mat
processing: PS_2216_z34_pan.mat
processing: HP_0522_z16_pan.mat
processing: PS_2216_z33_pan.mat
processing: HP_0508_z23_pan.mat
processing: PD_2084_z21_tum.mat
processing: HP_0522_z15_pan.mat
processing: PS_2216_z35_pan.mat
processing: PD_2127_z18_tum.mat
processing: PS_2250_z25_pan.mat
processing: HP_1002_z31_pan.mat
processing: HP_1002_z34_pan.mat
processing: PD_2127_z15_pan.mat
processing: PS_2250_z28_pan.mat
processing: PD_2127_z19_tum.mat
processing: HP_0726_z34_pan.mat
processing: HP_1002_z26_pan.mat
processing: HP_0726_z33_pan.mat
processing: HP_2182_z25_pan.mat
processing: PD_2158_z26_pan.mat
processing: HP_2182_z27_pan.mat
processing: PD_2158_z28_both.mat
processing: HP_1009_z26_pan.mat
processing: HP_1009_z22_pan.mat
processing: HP_2182_z31_pan.mat
processing: PS_2255_z24_pan.mat
processing: HP_2182_z24_pan.mat
process

In [8]:
# load as array
X_valid_ts = np.zeros([0,IN_DIM])
Y_valid_ts = np.zeros([0,IN_DIM])
X_valid_pl = np.zeros([0,IN_DIM])
Y_valid_pl = np.zeros([0,IN_DIM])
"""

"""
for j in range(test_ind,test_ind+1):
    foldername = os.path.join(mainfolder,str(j))
    case_id = os.listdir(foldername)
    for i in case_id:
        print('processing:',os.path.join(i)) 
        h5file = h5py.File(os.path.join(foldername,i),'r')
        if np.array(h5file['c_ts']).ndim is 1:
            continue
        kv = np.array(h5file['kv'],dtype=np.float32).transpose(1,0)

        C_ts =  np.concatenate((np.array(h5file['c_ts'],dtype=np.float32).transpose(1,0),
                                np.array(h5file['c_bds'],dtype=np.float32).transpose(1,0)),axis=0)
        C_ts_lbl=C_ts
        
        C_bd =  np.mean(np.array(h5file['c_pl_ori'],dtype=np.float32).transpose(1,0),axis=0,keepdims=True)
        C_bd_lbl =  C_bd
        norm_constant = np.max(C_bd_lbl)*(1-hct)

        # generate downsampled R1
        C_ts_ds,C_ts,bundle = gen_ds_R1(C_ts,INIT_DIM,C_bd_lbl,OUT_DIM=OUT_DIM,sample_pattern=sample_pattern,average=False)
        # generate blood input 
        C_bd_ds,C_bd,bundle = gen_ds_R1(C_bd,INIT_DIM,C_bd_lbl,OUT_DIM=OUT_DIM,sample_pattern=sample_pattern,average=False)
        # clip label 
        _,C_ts_lbl,_ = gen_ds_R1(C_ts_lbl,INIT_DIM,C_bd_lbl,OUT_DIM=OUT_DIM,sample_pattern=sample_pattern,average=False)
        _,C_bd_lbl,_ = gen_ds_R1(C_bd_lbl,INIT_DIM,C_bd_lbl,OUT_DIM=OUT_DIM,sample_pattern=sample_pattern,average=False)
        # generate spike
        C_ts_itnpl = gen_inge_spike(C_ts_ds,bundle)
        C_bd_itnpl = gen_inge_spike(C_bd_ds,bundle)

        bd_temp = np.repeat(C_bd_itnpl.reshape([-1,OUT_DIM]),C_ts.shape[0],axis=0)
        bd_temp2 = np.repeat(C_bd.reshape([-1,OUT_DIM]),C_ts.shape[0],axis=0)
    
        X_valid_ts = np.append(X_valid_ts,C_ts_itnpl/norm_constant,axis=0)
        Y_valid_ts = np.append(Y_valid_ts,C_ts/norm_constant,axis=0)
        X_valid_pl = np.append(X_valid_pl,bd_temp/norm_constant,axis=0)
        Y_valid_pl = np.append(Y_valid_pl,bd_temp2/norm_constant,axis=0)
        h5file.close()
        
print('Validation data loaded. X_valid_ts Shape:',X_valid_ts.shape)


<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-8-ef2d7de623a9>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if np.array(h5file['c_ts']).ndim is 1:


processing: HP_0504_z21_pan.mat
processing: HP_2245_z25_pan.mat
processing: PD_0205_z19_pan.mat
processing: HP_0504_z18_pan.mat
processing: PS_2204_z19_pan.mat
processing: PD_0205_z33_both.mat
processing: PS_2204_z23_pan.mat
processing: PS_2204_z26_pan.mat
processing: PS_2204_z24_pan.mat
processing: HP_2245_z29_pan.mat
processing: HP_0504_z22_pan.mat
Validation data loaded. X_valid_ts Shape: (9320, 130)


In [9]:
def gen_series():
    for __x, __y in  zip(X_train_ts, Y_train_ts):
        yield {"tissue_ds":__x}, {"Tissue":__y}
def gen_series_mul_inputs():
    for __x, __x2, __y in  zip(X_train_ts, X_train_pl, Y_train_ts):
        yield {"tissue_ds":__x, "aorta_fs":__x2}, {"Tissue":__y}
def gen_series_mul_inputs_mul_outputs():
    for __x, __x2, __y, __y2 in  zip(X_train_ts, X_train_pl, Y_train_ts, Y_train_pl):
        yield {"tissue_ds":__x, "aorta_fs":__x2}, {"Tissue":__y,"AIF":__y2}  
def gen_series_mul_inputs_3_outputs():
    for __x, __x2, __y, __y2, __y3 in  zip(X_train_ts, X_train_bd, Y_train_ts, Y_train_bd,Y_train):
        yield {"tissue_ds":__x, "aorta_fs":__x2}, {"Tissue":__y,"AIF":__y2,"kv":__y3}    

In [10]:
# construct simulation dataset
train_ds = tf.data.Dataset.from_generator(gen_series_mul_inputs_mul_outputs,output_types=({"tissue_ds":tf.float32,"aorta_fs":tf.float32},
                                                                              {"Tissue":tf.float32,"AIF":tf.float32}),
                                          output_shapes=({"tissue_ds":(IN_DIM),"aorta_fs":(IN_DIM)}, {"Tissue":(IN_DIM),"AIF":(IN_DIM)}))

train_ds = (
    train_ds
    .shuffle(X_train_ts.shape[0])
    .repeat()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

valid_ds = tf.data.Dataset.from_tensor_slices(({"tissue_ds":X_valid_ts,"aorta_fs":X_valid_pl},
                                               {"Tissue":Y_valid_ts,"AIF":Y_valid_pl}))
valid_ds = valid_ds.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

# number of steps per epoch
N_STEPS = np.ceil(Y_train_ts.shape[0]/BATCH_SIZE)

In [9]:
# ----------- pre-trained model -----------

prefix_pretrain = "Simulation_Train_Simulation_Valid_len%s_tofts_kvloss_noAIFnoise_breast_linear_ELU_corr_dim_smaller_ve_lim_ve_L1_MPAE_retrain_transformer_dense_standardTofts_fast_checkdata_relusigmoid"%(str(LENGTH))

model_path = f"/hdd1/chaowei/dce-dl/checkpoint/Q-kv-ssl-kvloss-sim-noAIFnoise/{prefix_pretrain}"

"""
model definition
"""
from tensorflow import keras
from tensorflow.keras import layers

def fake_loss(x,y):
    return 1e-10
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError

# Custom mixed loss function
def mixed_loss(y_true, y_pred, alpha=0.5):
    l1_loss = MeanAbsoluteError()(y_true, y_pred)
    mape_loss = MeanAbsolutePercentageError()(y_true, y_pred)
    
    # Combine the two losses with a weight factor (alpha)
    return alpha * l1_loss + (1 - alpha) * mape_loss

# Create a custom loss function with specific alpha
def custom_mixed_loss(alpha):
    def loss(y_true, y_pred):
        return mixed_loss(y_true, y_pred, alpha)
    return loss

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'rate': self.rate,
        })
        return config

embed_dim = 64
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
transformer_layer = 4
transformer_blocks = []
    
# ----------- construct model -----------
import datetime
from tensorflow.keras.regularizers import l2
tis_inp = tf.keras.Input(shape=(IN_DIM), name="Tissue")
f_bd = tf.keras.Input(shape=(IN_DIM), name="AIF")

with tf.name_scope("concat2"):
    x2 = tf.expand_dims(tis_inp,axis=-1)
    x3 = tf.expand_dims(f_bd,axis=-1)
    x = tf.keras.layers.Concatenate(axis=-1)([x2, x3])
    x = tf.keras.layers.Dense(embed_dim)(x)

with tf.name_scope("deconv"):
    for i in range(transformer_layer):
        transformer_blocks.append(TransformerBlock(embed_dim, num_heads, ff_dim))
        x = transformer_blocks[i](x)


with tf.name_scope("pred"): 
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    p_kv = tf.keras.layers.Dense(units=OUT_DIM,activation='sigmoid',name="KV")(x)

#     f_pd = con_tofts_tf(f_bd,p_kv[0,0],p_kv[0,1],p_kv[0,2],pad_num=IN_DIM-1)

model_pretrain = tf.keras.Model(
    inputs=[tis_inp,f_bd], outputs=[p_kv]
)

model_pretrain.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LR_RATE),loss=custom_mixed_loss(0.998))

model_path = os.path.join(model_path,"cp-{epoch:04d}.ckpt")
cp_path = os.path.dirname(model_path)
model_pretrain.load_weights(tf.train.latest_checkpoint(cp_path))

for layer in model_pretrain.layers:
    layer.trainable = False

In [10]:
# ----------- construct model -----------
import datetime
from tensorflow.keras.regularizers import l2


tis_inp = tf.keras.Input(shape=(IN_DIM), name="tissue_ds")
bd_inp = tf.keras.Input(shape=(IN_DIM), name="aorta_fs")

with tf.name_scope("upsample"):
    x2 = tf.keras.layers.Dense(units=IN_DIM)(tis_inp)
    x2 = tf.keras.layers.Dense(units=IN_DIM,activation='relu')(x2)
    f_ts = tf.keras.layers.Dense(units=IN_DIM,name="Tissue",activation='relu')(x2)
with tf.name_scope("upsample2"):
    x3 = tf.keras.layers.Dense(units=IN_DIM)(bd_inp)
    x3 = tf.keras.layers.Dense(units=IN_DIM,activation='relu')(x3)
    f_bd = tf.keras.layers.Dense(units=IN_DIM,name="AIF",activation='relu')(x3)
    
    
with tf.name_scope("pred"): 
    p_kv = model_pretrain([f_ts,f_bd])

model = tf.keras.Model(
    inputs=[tis_inp,bd_inp], outputs=[f_ts,f_bd]
)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LR_RATE),loss=tf.keras.losses.MeanSquaredError())
model.add_loss(DCloss_mag(f_ts, tis_inp,reg=0.1))
model.add_loss(DCloss_mag(f_bd, bd_inp,reg=0.1))
model.add_loss(o_Toftsloss(p_kv,f_ts,f_bd,reg=0.1,pad_num=IN_DIM-1,mode='VARPRODC',c_ts_inp=tis_inp,del_t=2))

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tissue_ds (InputLayer)          [(None, 130)]        0                                            
__________________________________________________________________________________________________
aorta_fs (InputLayer)           [(None, 130)]        0                                            
__________________________________________________________________________________________________
dense_34 (Dense)                (None, 130)          17030       tissue_ds[0][0]                  
__________________________________________________________________________________________________
dense_36 (Dense)                (None, 130)          17030       aorta_fs[0][0]                   
____________________________________________________________________________________________

In [ ]:


# ----------- train model -----------
# tensorboard callbck
logdir = os.path.join("/hdd1/chaowei/dce-dl/log/T1W-XAIFinp",prefix, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# save checkpoints
checkpoint_path = os.path.join("/hdd1/chaowei/dce-dl/checkpoint/T1W-XAIFinp",prefix,"cp-{epoch:04d}.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=CKPT_PERIOD)

# adaptive learning rate
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=10, min_lr=0)

if ISTRAIN is True:
    history = model.fit(train_ds,epochs=N_EPOCHS,steps_per_epoch=N_STEPS,validation_data=valid_ds,
                        callbacks = [reduce_lr, tensorboard_callback, cp_callback])


    print("completed")
    
    

Test model ...

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

if ISTRAIN is True:
    plt.figure(figsize=(16,6))
    plot_graphs(history, 'loss')